# Глава 19. Расширенные возможности функций

## Концепции проектирования функций

**Связность** - как разложить задачу на функции

**Взаимодействие** - как должны взаимодействовать функции

* **Взаимодействие: для передачи значений функции используйте аргументы, для возврата результатов - инструкцию `return`**


* **Взаимодействие: используйте глобальные переменные, только если это действительно необходимо**. Глобальные переменные могут порождать зависимости и проблемы согласованности, которые существенно осложняют отладку программ.


* **Взаимодействие: не воздействуйте на изменяемые аргументы, если вызывающая программа не предполагает этого**


* **Связность: каждая функция должна иметь единственное назначение**. Хорошо спроектированная функция должна решать одну задачу, которую можно выразить в одном повествовательном предложении (без широкого толкования и союзов)


* **Размер: каждая функция должна иметь относительно небольшой размер**


* **Взаимодействие: избегайте непосредственного изменения переменных в другом модуле**. Всегда, когда это возможно, для изменения переменных модуля вместо прямых инструкций используйте функции доступа.

Из приведенных выше правил проектирования есть свои исключения, включая те, что связаны с поддержкой ООП.

## Рекурсивные функции

**Рекурсивные функции** - функции, которые могут вызывать сами себя, прямо или косвенно, образуя цикл.

Это достаточно полезный прием, позволяющий реализовать обход структур данных с произвольной и неизвестной заранее организацией, также рекурсия является альтернативой простым циклам и итерациям, хотя и не обязательно более простой или более производительной.

### Вычисление суммы с применением рекурсии

In [1]:
def mysum(L):
    if not L:
        return 0
    else:
        return L[0] + mysum(L[1:])  # вызывает себя саму

mysum([1, 2, 3, 4, 5])

15

Когда рекурсия используется, как показано здесь, на каждом уровне
рекурсии для функции создается собственная копия локальной области видимости на стеке вызовов – в данном случае это означает, что на каждом уровне создается собственная переменная `L`.

### Альтернативные решения

In [2]:
# трехместный оператор
def mysum(L):
    return 0 if not L else L[0] + mysum(L[1:])

# суммирует любые типы, предполагает наличие хотя бы одного значения
def mysum(L):
    return L[0] if len(L) == 1 else L[0] + mysum(L[1:])

# использует расширенную операцию присваивания последовательностей
def mysum(L):
    first, *rest = L
    return first if not rest else first + mysum(rest)

Последние две функции будут завершаться с ошибкой при получении пустого списка, но они позволяют находить сумму последовательностей не только чисел, но и объектов любых типов, поддерживающих операцию `+`

In [3]:
mysum([1])

1

In [4]:
mysum(['spam', 'ham', 'eggs'])

'spamhameggs'

In [5]:
mysum('spam')  # строка - это последовательность односимвольных строк

'spam'

**Рекурсия может быть прямой**, как в примерах выше, **или косвенной**, как в следующем примере (когда функция вызывает другую функцию, которая в свою очередь вызывает функцию, вызвавшую ее).

Конечный результат будет тем же самым, только в этом случае на каждом уровне рекурсии будет выполняться два вызова функций вместо одного:

In [6]:
def mysum(L):
    if not L:
        return 0
    return nonempty(L)  # вызов функции, которая вызовет эту функцию

def nonempty(L):
    return L[0] + mysum(L[1:])  # косвенная рекурсия

mysum([1.1, 2.2, 3.3, 4.4, 5.5])

16.5

### Инструкции циклов вместо рекурсии

Для случая выше рекурсия - слишком тяжеловесный механизм

Инструкция `for`, выполняя итерации автоматически, в большинстве случаев позволяет избавиться от рекурсии (которая обычно менее эффективна с  точки зрения использования памяти и скорости выполнения):

In [7]:
L = [1, 2, 3, 4, 5]

sum = 0
for x in L: sum += x
sum

15

При использовании инструкций циклов отпадает необходимость создавать для каждой итерации копии локальной области видимости в стеке вызовов и ликвидируются потери времени, необходимые на вызов функции.

### Обработка произвольных структур данных

С другой стороны, рекурсия (или эквивалентные ей алгоритмы, основанные на использовании стека, которые мы рассмотрим здесь) может оказаться востребованной для реализации обхода структур данных с  произвольной организацией.

В качестве простого примера, который поможет оценить роль рекурсии
в данном контексте, рассмотрим задачу вычисления суммы всех чисел в структуре, состоящей из вложенных списков, как показано ниже:

In [ ]:
L = [1, [2, [3, 4], 5], 6, [7, 8]] # Произвольно вложенные списки

Простые инструкции циклов в этом случае не годятся, потому что выполнить обход такой структуры с помощью линейных итераций не удастся. Вложенные инструкции циклов также не могут использоваться, потому что списки могут иметь произвольные уровни вложенности и  иметь произвольную организацию. Вместо этого следующий пример выполняет обход вложенных списков с помощью рекурсии:

In [8]:
def sumtree(L):
    tot = 0
    for x in L: # Обход элементов одного уровня
        if not isinstance(x, list):
            tot += x  # Числа суммируются непосредственно
        else:
            # Списки обрабатываются рекурсивными вызовами
            tot += sumtree(x) 
    return tot

In [9]:
sumtree(L)

15

In [10]:
# Патологические случаи
# Выведет 15 (центр тяжести справа)
print(sumtree([1, [2, [3, [4, [5]]]]]))

15


In [11]:
# Выведет 15 (центр тяжести слева)
print(sumtree([[[[[1], 2], 3], 4], 5]))

15


## Функции – это объекты: атрибуты и аннотации

### Косвенный вызов функций

Объекты функций могут присваиваться, передаться другим функциям, сохраняться в  структурах данных и  так далее, как если бы они были простыми числами или строками.

Кроме того, объекты функций поддерживают специальные операции: они могут вызываться перечислением аргументов в круглых скобках, следующих сразу же за выражением функции. И тем не менее, функции принадлежат к категории обычных объектов.

Поскольку аргументы передаются путем присваивания объектов, функции
легко можно передавать другим функциям в  виде аргументов. В  результате вызываемая функция может вызвать переданную ей функцию простым добавлением списка аргументов в круглых скобках:

In [12]:
def echo(message):
    print(message)
    
def indirect(func, arg):
    func(arg)  # вызов объекта добавлением ()
    
indirect(echo, 'Argument call!')  # передача функции в функцию

Argument call!


Существует даже возможность наполнять структуры данных функциями, как
если бы они были простыми числами или строками. В этом нет ничего необычного, так как составные типы объектов могут содержать объекты любых типов:

In [13]:
schedule = [(echo, 'Spam!'), (echo, 'Ham!')]

for func, arg in schedule:
    func(arg) # Вызов функции, сохраненной в контейнере

Spam!
Ham!


Функции могут также создаваться и возвращаться другими функциями

In [14]:
def make(label):  # создает функцию, но не вызывает ее
    def echo(message):
        print(label + ':' + message)
    return echo

F = make('Spam')  # метка сохраняется во вложенной области видимости
F('Ham!')

Spam:Ham!


In [15]:
F('Eggs!')

Spam:Eggs!


### Интроспекция функций

Выражение вызова  – это лишь одна из операций, которые могут применяться к объектам функций. Кроме того, мы можем получить базовый доступ к  атрибутам функции:

In [16]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Механизмы интроспекции позволяют нам также исследовать детали реализации – каждая функция, например, имеет присоединенный к ней объект с программным кодом, из которого можно получить такие сведения, как список локальных переменных и аргументов:

In [17]:
func.__code__

<code object echo at 0x7f2ca058f8a0, file "<ipython-input-12-d55ef35e4539>", line 1>

In [18]:
dir(func.__code__)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_stacksize',
 'co_varnames']

In [19]:
func.__code__.co_varnames

('message',)

In [20]:
func.__code__.co_argcount

1

### Атрибуты функций

In [21]:
func.count = 0
func.count += 1
func.count

1

In [22]:
func.handles = 'Button-Press'
func.handles

'Button-Press'

In [23]:
dir(func)[-5:]

['__sizeof__', '__str__', '__subclasshook__', 'count', 'handles']

Как было показано в указанной главе, такие атрибуты можно использовать для хранения информации о состоянии непосредственно в объекте функции и отказаться от использования других приемов, таких как применение глобальных или нелокальных переменных и классов.

В отличие от нелокальных переменных, атрибуты функций доступны в любом месте программы, где доступна сама функция. В некотором смысле атрибуты можно рассматривать, как имитацию «статических локальных переменных», имеющихся в других языках программирования, – переменных, которые являются локальными для функции, но сохраняют свои значения после выхода из функции. Атрибуты связаны с объектами, а не с областями видимости, но конечный эффект от их использования получается тот же.

### Аннотации функций в версии 3.0

В Python 3.0 (но не в 2.6) имеется также возможность присоединять к объектам функций **краткое описание (аннотацию)** – произвольные данные об аргументах функции и  о возвращаемом значении.

Для создания аннотаций в  языке Python используется специальный синтаксис, но интерпретатор не выполняет никаких операций с ними – аннотации совершенно необязательны; если они присутствуют, они просто сохраняются в  атрибутах `__annotations__` объектов функций и могут использоваться другими инструментами.

* **Аннотации для аргументов** указываются через двоеточие, сразу после имени аргумента.


* **Для возвращаемого значения** – после символов `->`, вслед за списком аргументов

In [24]:
def func(a: 'spam', b: (1, 10), c: float) -> int:
    return a + b + c

func(1, 2, 3)

6

In [25]:
func.__annotations__

{'a': 'spam', 'b': (1, 10), 'c': float, 'return': int}

**В аннотированных аргументах все еще можно указывать значения по умолчанию** – аннотация (и символ `:`) находится перед значением по умолчанию (и перед символом `=`).

В следующем примере фрагмент `a: 'spam' = 4` означает, что аргумент `a` по умолчанию получает значение `4` и аннотирован строкой `'spam'`:

In [26]:
def func(a: 'spam' = 4, b: (1, 10) = 5, c: float = 6) -> int:
    return a + b + c

In [27]:
func()  # все аргументы получают значения по умолчанию

15

In [28]:
func(1, c=10)  # именованные аргументы действуют как обычно

16

## Анонимные функции: `lambda`

Подобно инструкции `def` **выражение `lambda`** создает функцию, которая будет вызываться позднее, но в отличие от инструкции `def`, выражение возвращает функцию, а не связывает ее с  именем. Именно поэтому `lambda`-выражения иногда называют **анонимными (то есть безымянными) функциями**.

На практике они часто используются, как способ получить встроенную функцию или отложить выполнение фрагмента программного кода.

### Основы `lambda`-выражений

В общем виде `lambda`-выражение состоит из ключевого слова `lambda`, за которым следуют один или более аргументов (точно так же, как список аргументов в круглых скобках в заголовке инструкции `def`) и далее, вслед за двоеточием, находится выражение:

`lambda arg1, arg2, ..., argN: выражение, использующее аргументы`

В качестве результата `lambda`-выражения возвращают точно такие же объекты функций, которые создаются инструкциями `def`, но здесь есть несколько различий, которые делают `lambda`-выражения удобными в некоторых специализированных случаях:

* **`lambda` - это выражение, а не инструкция**. По этой причине ключевое слово `lambda` может появляться там, где синтаксис не позволяет использовать инструкцию `def`, - внутри литералов или в вызовах функций, например. Кроме того lambda-выражение возвращает значение (новую функцию), которое при желании можно присвоить переменной.


* **Тело `lambda` - это не блок инструкций, а единственное выражение**. `lambda`-выражения предназначены для создания простых функций, а инструкции `def` - для решения более сложных задач.

In [29]:
def func(x, y, z): return x + y + z

# аналогичное lambda-выражение

f = lambda x, y, z: x + y + z

func(1, 2, 3) == f(1, 2, 3)

True

> В `lambda`-выражениях точно так же можно использовать аргументы со значениями по умолчанию:

In [30]:
x = (lambda a='fee', b='fie', c='foe': a + b + c)
x()

'feefiefoe'

Для `lambda`-выражений используются те же самые правила поиска переменных в областях видимости, что и для вложенных инструкций `def`.

`lambda`-выражения создают локальную область видимости, как и вложенные инструкции `def`, и автоматически получают доступ к  именам в объемлющих функциях, в модуле и во встроенной области видимости (в соответствии с правилом LEGB):

In [31]:
def knights():
    title = 'Sir'
    action = (lambda x: title + ' ' + x)
    return action

act = knights()
act('Robin')

'Sir Robin'

### Когда можно использовать `lambda`-выражения?

Вообще говоря, `lambda`-выражения очень удобны для создания очень маленьких функций, к  тому же они позволяют встраивать определения функций в программный код, который их использует. Они не являются предметом первой необходимости (вы всегда сможете вместо них использовать инструкции `def`), но они позволяют упростить сценарии, где требуется внедрять небольшие фрагменты программного кода.

`lambda`-выражения также часто используются для создания таблиц переходов, которые представляют собой списки или словари действий, выполняемых по требованию. Например:

In [32]:
L = [lambda x: x**2, # Встроенные определения функций
     lambda x: x**3,
     lambda x: x**4] # Список из трех функций

for f in L:
    print(f(2))

4
8
16


В действительности, подобные таблицы действий в языке Python можно создавать с помощью словарей и других структур данных. Ниже приводится другой пример, выполненный в интерактивном сеансе:

In [33]:
key = 'got'

{'already': (lambda: 2 + 2),
 'got': (lambda: 2 * 4),
 'one': (lambda: 2 ** 6)}[key]()

8

**Близость программного кода**, которую обеспечивают `lambda`-выражения, особенно полезна, когда функции используются в единственном месте – если три функции в этом фрагменте не используются где-то еще, определенно имеет смысл встроить их в определение словаря в виде `lambda`-выражений.

Кроме того, инструкции `def` требуют даже для маленьких функций указывать имена, а они могут вступить в конфликт с другими именами в файле модуля (возможно, хотя и маловероятно).

`lambda`-выражения также очень удобно использовать в  списках аргументов функций – для определения временных функций, которые больше нигде в программе не используются, – мы увидим примеры такого использования ниже, в этой главе, когда будем изучать функцию `map`.

### Как (не) запутать программный код на языке Python

Трехместные операторы внутри `lambda`

In [34]:
lower = (lambda x, y: x if x < y else y)
lower('aa', 'bb')

'aa'

In [35]:
lower(5, 2)

2

Кроме того, если внутри `lambda`-выражения потребуется выполнять циклы, их можно заменить вызовами функции `map` и генераторами списков

In [36]:
import sys
showall = lambda x: list(map(sys.stdout.write, x))

t = showall(['spam\n', 'toast\n', 'eggs\n'])

spam
toast
eggs


### Вложенные `lambda`-выражения и области видимости

`lambda`-выражения чаще других используют возможность поиска в области видимости вложенной функции (символ `E` в названии правила `LEGB`).

Например, следующее ниже `lambda`-выражение находится внутри инструкции `def` – типичный случай  – и  потому получает значение имени `x` из области видимости объемлющей функции, имевшееся на момент ее вызова:

In [37]:
def action(x):
    return (lambda y: x + y)  # создать и вернуть ф-ию, запомнить x

In [38]:
act = action(99)
act

<function __main__.action.<locals>.<lambda>(y)>

In [39]:
act(2)

101

`lambda`-выражения обладают доступом к именам во всех объемлющих `lambda`-выражениях. Это сложно себе вообразить, но представьте, что мы записали предыдущую инструкцию `def` в виде `lambda`-выражения:

In [40]:
action = (lambda x: (lambda y: x + y))
act = action(99)
act(3)

102

In [41]:
((lambda x: (lambda y: x + y))(99))(4)

103

Эта структура `lambda`-выражений создает функцию, которая при вызове создает другую функцию. В  обоих случаях вложенное `lambda`-выражение имеет доступ к переменной `x` в объемлющем `lambda`-выражении. Этот фрагмент будет
работать, но программный код выглядит весьма замысловато, поэтому в интересах соблюдения удобочитаемости лучше избегать использования вложенных друг в друга `lambda`-выражений.

## Отображение функций на последовательности: `map`

**Функция `map`** применяет указанную функцию к  каждому элементу последовательности и возвращает итерируемый объект, содержащий результаты всех вызовов функции.

In [42]:
def inc(x): return x + 10  # функция, которая должна быть вызвана

list(map(inc, [1, 2, 3, 4, 5]))

[11, 12, 13, 14, 15]

Функция `map` ожидает получить в  первом аргументе функцию, поэтому здесь часто можно встретить `lambda`-выражения:

In [43]:
list(map(lambda x: x + 10, range(1, 6)))

[11, 12, 13, 14, 15]

Такой вариант использования функции `map` представляет собой эквивалент цикла `for`, поэтому, написав несколько строк, эту утилиту в общем виде вы можете реализовать самостоятельно:

In [44]:
def mymap(func, seq):
    res = []
    for x in seq: res.append(func(x))
    return res

In [45]:
list(map(inc, range(5)))

[10, 11, 12, 13, 14]

In [46]:
mymap(inc, range(5))

[10, 11, 12, 13, 14]

Однако функция `map` является встроенной функцией, поэтому она доступна всегда, всегда работает одним и  тем же способом и  обладает некоторыми
преимуществами производительности

>**При передаче нескольких последовательностей** функция `map` предполагает, что ей будет передана функция, принимающая N аргументов для N последовательностей.

Здесь функция `pow` при каждом вызове принимает от функции `map` два
аргумента – по одному из каждой последовательности.

In [47]:
list(map(pow, [1, 2, 3], [2, 3, 4]))

[1, 8, 81]

Этот вызов функции `map` напоминает генераторы списков, которые рассматривались в  главе  14, и  с которыми мы встретимся еще раз в  следующей главе.

Основное отличие состоит в том, что `map` применяет к каждому элементу последовательности не произвольное выражение, а функцию. Вследствие этого ограничения она обладает меньшей гибкостью. Однако, современная реализация
`map` в некоторых случаях обладает более высокой производительностью, чем генераторы списков (например, когда отображается встроенная функция), и использовать ее проще.

## Средства функционального программирования: `filter` и `reduce`

Родственные `map` функции отфильтровывают элементы с помощью функций, выполняющих проверку (`filter`), и применяют функции к парам элементов, накапливая результаты (`reduce`).

Например, следующий вызов функции `filter` отбирает элементы последовательности больше нуля:

In [48]:
list(filter((lambda x: x > 0), range(-5, 5)))

[1, 2, 3, 4]

**Функция `reduce`** в 3.0 была перемещена в **модуль `functools`**. Она принимает итератор, но сама возвращает не итератор, а  одиночный объект.
Ниже приводятся два вызова функции `reduce`, которые вычисляют сумму
и произведение элементов списка:

In [49]:
from functools import reduce

reduce((lambda x, y: x + y), [1, 2, 3, 4])

10

In [50]:
reduce((lambda x, y: x * y), [1, 2, 3, 4])

24

На каждом шаге функция `reduce` передает текущую сумму или произведение
вместе со следующим элементом списка `lambda`-функции. По умолчанию первый элемент последовательности принимается в  качестве начального значения.

`reduce` может принимать третий необязательный аргумент, который используется в  качестве начального значения и  служит значением по умолчанию, когда передаваемая последовательность не содержит ни одного элемента

In [51]:
reduce((lambda x, y: x * y), [1, 2, 3, 4], 0)

0